## Le seuillage d'images (Images thresholding)

### <span style=color:green> Seuillage d'une en image en niveau de gris </span>

La segmentation par seuillage est une binarisation de l'image. Elle divise donc l'image en deux classes et est appliquée sur une image en niveau de gris.

Pour choisir le seuil de segmentation, une analyse préable de l'histogramme et/ou des statisques sur l'image est nécessaire afin de répérer les différents modes. Le choix du seuil peut être difficile dans certains cas, on peut alors tester plusieurs valeurs ou faire une observation sur l'image pour répérer les tons de gris sombres et clairs.

1. **le seuillage global** : le choix du seuil est facile car l'histogramme présente deux modes assez disctincts.
2. **le seuillage automatique d'Otsu** : L'image ne contient que deux classes (avant-plan et arrière-plan) puis l'algorithme calcule le seuil optimal en minimisant leur variance pondérée intra-classe. Autrement dit, Otsu cherche le seuil qui rend les pixels de chaque classe aussi similaires que possible. Pour chaque classe, la variance intra-classe mesure à quel point les valeurs de gris des pixels de cette classe sont étroitement regroupées autour de la moyenne de cette classe. Otsu ne détermine que le seuil optimal pour séparer les deux classes de pixels et doit donc être utilisée conjointement avec un type de seuillage spécifique pour appliquer ce seuil.
3. **le seuillage multiple** : 3 modes ou plus distincts sont visibles sur l'histogramme, on a plus de deux classes. on peut utiliser l'algorithme multi Otsu.
4. **le seuillage adaptatif** : efficace pour ombrage et éclairage non uniforme.

In [19]:
# import modules
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import threshold_multiotsu

In [23]:
def display_histogram(image_name: str, _xlim=255, _ylim=2500) -> None:
    """ Displays the histogram of an image """
    # Reading, checking and display image
    image = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Error: image could not be read. Check file path: {image_name}")
    
    # Calculate and Plotting the histogram
    image_hist = cv.calcHist([image], [0], None, [256], [0, 256])
    
    # Set the figure and plotting result  
    _, axes = plt.subplots(ncols=2, figsize=(12, 3))
    axe = axes.ravel()
    axe[0].imshow(image, cmap="gray")
    axe[0].set_title("Original")
    axe[0].axis("off") # to provide a better display we hide axes but we delete size information

    axe[1].plot(image_hist)
    axe[1].set_xlim([0, _xlim]) # to provide a better display we limit x and y axes values
    axe[1].set_ylim([0, _ylim])
    axe[1].set_xlabel('Pixels value or bins')
    axe[1].set_ylabel('Occurences')
    
    plt.subplots_adjust()
    plt.show()


plant = "Images/plant.png" 
#display_histogram(plant) 

In [14]:
def display_threshold(image_name: str, thresh=60) -> None:
    """ Image global Thresholding """
    # Reading, checking and display image
    image = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Error: image could not be read. Check file path: {image_name}")
    
    # Calculate the histogram
    image_hist = cv.calcHist([image], [0], None, [256], [0, 256])
    # Perform global thresholding
    _, thresholded = cv.threshold(image, thresh, 255, cv.THRESH_BINARY)
    
    # Set the figure  and plotting results 
    _, axes = plt.subplots(ncols=3, figsize=(12, 3))
    axe = axes.ravel()
    axe[0].imshow(image, cmap="gray")
    axe[0].set_title("Original")
    axe[0].axis("off") # to provide a better display we hide axes
    
    axe[1].plot(image_hist)
    axe[1].set_title("Histogram")
    axe[1].set_xlabel('Pixels value or bins')
    axe[1].set_ylabel('Occurences')
    axe[1].axvline(thresh, color='r')
    
    axe[2].imshow(thresholded, cmap="gray")
    axe[2].set_title("Thresholded")
    axe[2].axis("off")
    
    plt.subplots_adjust()
    plt.show()

#display_threshold(plant, 25)

In [18]:
def display_otsu_threshold(image_name: str) -> float:
    """ Otsu Thresholding """
    # Reading, checking and display image
    image = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Error: image could not be read. Check file path: {image_name}")
    
    # Calculate the histogram
    image_hist = cv.calcHist([image], [0], None, [256], [0, 256])
    # Perform Otsu thresholding
    threshold, otsu_thresholded = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    # Set the figure  and plotting results 
    _, axes = plt.subplots(ncols=3, figsize=(12, 3))
    axe = axes.ravel()
    axe[0].imshow(image, cmap="gray")
    axe[0].set_title("Original")
    axe[0].axis("off") # to provide a better display we hide axes
    
    axe[1].plot(image_hist)
    axe[1].set_title("Histogram")
    axe[1].set_xlabel('Pixels value or bins')
    axe[1].set_ylabel('Occurences')
    axe[1].axvline(threshold, color='r')
    
    axe[2].imshow(otsu_thresholded, cmap="gray")
    axe[2].set_title("Otsu Thresholded")
    axe[2].axis("off")
    
    plt.subplots_adjust()
    plt.show()
    
    return f"Otsu threshold value: {threshold}"

#display_otsu_threshold(plant)

In [27]:
def display_multiOtsu_threshold(image_name: str) -> float:
    """ Otsu Thresholding """
    # Reading, checking and display image
    image = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Error: image could not be read. Check file path: {image_name}")
    
    # Calculate the histogram
    image_hist = cv.calcHist([image], [0], None, [256], [0, 256])
    # Perform Multi Otsu thresholding: generating three classes by default
    thresholds = threshold_multiotsu(image)

    # Using the threshold values, we generate the three regions.
    regions = np.digitize(image, bins=thresholds)
    
    # Set the figure  and plotting results 
    _, axes = plt.subplots(ncols=3, figsize=(12, 3))
    axe = axes.ravel()
    axe[0].imshow(image, cmap="gray")
    axe[0].set_title("Original")
    axe[0].axis("off") # to provide a better display we hide axes
    
    axe[1].plot(image_hist)
    axe[1].set_title("Histogram")
    axe[1].set_xlabel('Pixels value or bins')
    axe[1].set_ylabel('Occurences')
    for thresh in thresholds:
        axe[1].axvline(thresh, color='r')
    
    axe[2].imshow(regions, cmap="jet") # Try cmap=viridis or plasma, tab20, set1, accent
    axe[2].set_title("Multi Otsu Thresholded")
    axe[2].axis("off")
    
    plt.subplots_adjust()
    plt.show()
    
    return f"Multi Otsu threshold values: {thresholds}"

#display_multiOtsu_threshold(plant)

In [36]:
def display_adaptative_threshold(image_name: str) -> None:
    """ Adaptativ Thresholding """
    # Reading, checking and display image
    image = cv.imread(image_name, cv.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Error: image could not be read. Check file path: {image_name}")
    
    # Calculate the histogram
    image_hist = cv.calcHist([image], [0], None, [256], [0, 256])
    # Perform Otsu thresholding
    thresholded = cv.adaptiveThreshold(image, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 3, 4)
    
    # Set the figure  and plotting results 
    _, axes = plt.subplots(ncols=3, figsize=(12, 3))
    axe = axes.ravel()
    axe[0].imshow(image, cmap="gray")
    axe[0].set_title("Original")
    axe[0].axis("off") # to provide a better display we hide axes
    
    axe[1].plot(image_hist)
    axe[1].set_title("Histogram")
    axe[1].set_xlabel('Pixels value or bins')
    axe[1].set_ylabel('Occurences')
    
    axe[2].imshow(thresholded, cmap="gray")
    axe[2].set_title("Adaptative Thresholded")
    axe[2].axis("off")
    
    plt.subplots_adjust()
    plt.show()
    

#display_adaptative_threshold(plant)

### <span style=color:green> Seuillage d'image RGB </span>

Le seuillage se fait ici sur les différents ou sur l'un des canaux. L'espace RGB mélange à la fois la luminance et la chrominance donc un seuillage sur un canal ou plus altèrera la couleur. Il est alors préférable de travailler sur l'espace HSV ou YUV en travaillant sur le canal V.